In [1]:
import os
import numpy as np
from scipy.io import loadmat,savemat
import nibabel as nib
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch

In [23]:
nifti_path = r'/Users/kurtschilling/Data/harmonization/sA/A/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz'
mask_path = r'/Users/kurtschilling/Data/harmonization/sA/A/mask.nii.gz'
out_path = r'/Users/kurtschilling/Data/harmonization/sA/B/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz'


In [54]:
nifti_path = os.path.normpath(nifti_path)
img = nib.load(nifti_path)
data = img.get_fdata()
print(data.shape)

mask_path = os.path.normpath(mask_path)
mask_img = nib.load(mask_path)
mask = mask_img.get_fdata()
print(mask.shape)

out_path = os.path.normpath(out_path)
o_img = nib.load(out_path)
o_data = o_img.get_fdata()
print(o_data.shape)

(96, 96, 60, 15)
(96, 96, 60)
(96, 96, 60, 15)


In [78]:
num = np.count_nonzero(mask)
print(num)

X = np.zeros((num,15))
Y = np.zeros((num,15))
index = 0
for i in range(dims[0]):
    for j in range(dims[1]):
        for k in range(dims[2]):
            if (mask[i,j,k] == 1):
                
                temp_sh_ip = data[i,j,k,:]
                temp = np.array(temp_sh_ip)
                temp = temp.T
                
                temp_sh_ip2 = o_data[i,j,k,:]
                temp2 = np.array(temp_sh_ip2)
                temp2 = temp2.T
                #print(temp_sh_ip.shape)
                
                X[index,:] = temp
                Y[index,:] = temp2
                index += 1
                
print(X.shape)
print(Y.shape)
print(X.dtype)
print(Y.dtype)
            

84365
(84365, 15)
(84365, 15)
float64
float64


In [97]:
# remove NaN, Inf
from numpy import asarray as ar

arr1 = np.isnan(X).any(axis=1)
arr2 = np.isnan(Y).any(axis=1)
arr3 = np.isinf(X).any(axis=1)
arr4 = np.isinf(Y).any(axis=1)
arr5 = ar(arr1) | ar(arr2) | ar(arr3) | ar(arr4)
print(arr5.shape)
X = X[~arr5,:]
Y = Y[~arr5,:]
print(X.shape)
print(Y.shape)
            

(84318,)
(84318, 15)
(84318, 15)


In [99]:
# remove > 10
arr6 = np.greater(np.abs(X),10).any(axis=1)
arr7 = np.greater(np.abs(Y),10).any(axis=1)
arr8 = ar(arr6) | ar(arr7)
print(arr8)
print(arr8.shape)
X = X[~arr8,:]
Y = Y[~arr8,:]
print(X.shape)
print(Y.shape)

[False False False ... False False False]
(84318,)
(84241, 15)
(84241, 15)


In [105]:
X.astype(float)
Y.astype(float)

array([[ 3.31935108e-01, -1.62949245e-02, -1.75707005e-02, ...,
         1.30695011e-03,  1.53167825e-03, -3.45649128e-03],
       [ 4.22008395e-01, -1.61839109e-02,  1.75551195e-02, ...,
         5.78307966e-03, -4.86385310e-03, -7.69814593e-04],
       [ 4.09705073e-01, -2.72899829e-02, -5.67432866e-03, ...,
         2.61530280e-04, -8.84804875e-04,  4.88408084e-04],
       ...,
       [ 5.01835287e-01, -6.50745537e-03,  3.47751714e-02, ...,
        -1.56805990e-03, -1.27911568e-04,  6.38413196e-03],
       [ 4.42578048e-01, -3.79809029e-02, -8.91749002e-03, ...,
        -2.91226059e-03,  3.27009801e-03,  2.13469670e-04],
       [ 3.36586356e-01, -1.90302394e-02, -1.91237833e-02, ...,
         7.72789260e-03,  3.87520995e-03,  6.25888235e-04]])

In [110]:
from torch.utils.data import Dataset
from torchvision import transforms

class SHDataSet(Dataset):
    def __init__(self,X,Y):
        
        self.X = X
        self.Y = Y
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        
        #vec_a = np.reshape(self.X[i,:],(15, 1, 1))
        #vec_b = np.reshape(self.Y[i,:],(15, 1, 1))
        vec_a = np.reshape(self.X[i,:],(1, 15, 1))
        vec_b = np.reshape(self.Y[i,:],(1, 15, 1))
        a = self.to_tensor(vec_a)
        b = self.to_tensor(vec_b)
    
        return a,b
    
shset = SHDataSet(X,Y)
print(len(shset))


84241
84241


In [111]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1 * 15, 400)
        self.fc2 = nn.Linear(400,200)
        self.fc3 = nn.Linear(200,50)
        self.fc4 = nn.Linear(50,200)
        self.fc5 = nn.Linear(200,15)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
        
net = Net()
print(net)  

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#criterion = nn.MSELoss()


Net(
  (fc1): Linear(in_features=15, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=15, bias=True)
)


In [114]:
from __future__ import print_function, division
import torch


def train(model, device, loader, optimizer):
    model.train()

    correct = 0
    total_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        data, target = Variable(data).float(), Variable(target).float()
        
        optimizer.zero_grad()
        output = model(data)
        output = output.to(device)

        loss = criterion(output,target)
        total_loss += loss.item()
      
        loss.backward()
        optimizer.step()
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data),len(shset.X),100.*batch_idx/len(shset.X),loss.data[0]))

    avg_loss = total_loss / batch_idx
    print('\tTraining set: Average loss: {:.4f}'.format(avg_loss))
   
    return avg_loss




In [132]:
def test(model, device, loader):
    model.eval()

    correct = 0
    total_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        data, target = Variable(data).float(), Variable(target).float()

        output = model(data)
        output = output.to(device)

        loss = criterion(output, target)
        total_loss += loss.item()

    avg_loss = total_loss / batch_idx
    print('\tTesting set: Average loss: {:.4f}'.format(avg_loss))

    return avg_loss

In [133]:
# batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42
bs_size = 1000

# Creating data indices for training and validation splits:
dataset_size = len(shset)
print(dataset_size)

indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]




84241


In [134]:
from torch.utils.data.sampler import SubsetRandomSampler

use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(shset, batch_size=bs_size, sampler=train_sampler)
validation_loader = DataLoader(shset, batch_size=bs_size, sampler=valid_sampler)


In [135]:
import random

train_loss_file = 'train_loss_split.txt'
f = open(train_loss_file, 'w')
f.close()
validate_loss_file = 'validate_loss_split.txt'
f = open(validate_loss_file, 'w')
f.close()

model_file = 'saved_model_split'

In [137]:
for epoch in range(1, 51):
        print('\nEpoch %d: ' % epoch)
        loss = train(net, device, train_loader, optimizer)

        with open(train_loss_file, "a") as file:
            file.write(str(loss))
            file.write('\n')

        loss = test(net, device, validation_loader)

        with open(validate_loss_file, "a") as file:
            file.write(str(loss))
            file.write('\n')

        if epoch % 5 == 0:
            with open(model_file, 'wb') as f:
                torch.save(net.state_dict(), f)




Epoch 1: 
Train Epoch: 1 [0/84241 (0%)]	Loss: 0.016427
Train Epoch: 1 [1000/84241 (0%)]	Loss: 0.008770
Train Epoch: 1 [2000/84241 (0%)]	Loss: 0.007082
Train Epoch: 1 [3000/84241 (0%)]	Loss: 0.008833
Train Epoch: 1 [4000/84241 (0%)]	Loss: 0.008086


/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [5000/84241 (0%)]	Loss: 0.008809
Train Epoch: 1 [6000/84241 (0%)]	Loss: 0.009040
Train Epoch: 1 [7000/84241 (0%)]	Loss: 0.013558
Train Epoch: 1 [8000/84241 (0%)]	Loss: 0.008609
Train Epoch: 1 [9000/84241 (0%)]	Loss: 0.009770
Train Epoch: 1 [10000/84241 (0%)]	Loss: 0.009916
Train Epoch: 1 [11000/84241 (0%)]	Loss: 0.008928
Train Epoch: 1 [12000/84241 (0%)]	Loss: 0.008623
Train Epoch: 1 [13000/84241 (0%)]	Loss: 0.008034
Train Epoch: 1 [14000/84241 (0%)]	Loss: 0.007685
Train Epoch: 1 [15000/84241 (0%)]	Loss: 0.013070
Train Epoch: 1 [16000/84241 (0%)]	Loss: 0.007575
Train Epoch: 1 [17000/84241 (0%)]	Loss: 0.009359
Train Epoch: 1 [18000/84241 (0%)]	Loss: 0.007783
Train Epoch: 1 [19000/84241 (0%)]	Loss: 0.008132
Train Epoch: 1 [20000/84241 (0%)]	Loss: 0.008048
Train Epoch: 1 [21000/84241 (0%)]	Loss: 0.008099
Train Epoch: 1 [22000/84241 (0%)]	Loss: 0.008278
Train Epoch: 1 [23000/84241 (0%)]	Loss: 0.008159
Train Epoch: 1 [24000/84241 (0%)]	Loss: 0.008283
Train Epoch: 1 [25000/842

Train Epoch: 3 [37000/84241 (0%)]	Loss: 0.008014
Train Epoch: 3 [38000/84241 (0%)]	Loss: 0.008153
Train Epoch: 3 [39000/84241 (0%)]	Loss: 0.007043
Train Epoch: 3 [40000/84241 (0%)]	Loss: 0.007687
Train Epoch: 3 [41000/84241 (0%)]	Loss: 0.010396
Train Epoch: 3 [42000/84241 (0%)]	Loss: 0.007704
Train Epoch: 3 [43000/84241 (0%)]	Loss: 0.006567
Train Epoch: 3 [44000/84241 (0%)]	Loss: 0.010716
Train Epoch: 3 [45000/84241 (0%)]	Loss: 0.007117
Train Epoch: 3 [46000/84241 (0%)]	Loss: 0.008393
Train Epoch: 3 [47000/84241 (0%)]	Loss: 0.008880
Train Epoch: 3 [48000/84241 (0%)]	Loss: 0.007847
Train Epoch: 3 [49000/84241 (0%)]	Loss: 0.007148
Train Epoch: 3 [50000/84241 (0%)]	Loss: 0.008683
Train Epoch: 3 [51000/84241 (0%)]	Loss: 0.009688
Train Epoch: 3 [52000/84241 (0%)]	Loss: 0.007771
Train Epoch: 3 [53000/84241 (0%)]	Loss: 0.007277
Train Epoch: 3 [54000/84241 (0%)]	Loss: 0.008605
Train Epoch: 3 [55000/84241 (0%)]	Loss: 0.007337
Train Epoch: 3 [56000/84241 (0%)]	Loss: 0.013616
Train Epoch: 3 [5700

	Testing set: Average loss: 0.0084

Epoch 6: 
Train Epoch: 6 [0/84241 (0%)]	Loss: 0.007186
Train Epoch: 6 [1000/84241 (0%)]	Loss: 0.008275
Train Epoch: 6 [2000/84241 (0%)]	Loss: 0.007581
Train Epoch: 6 [3000/84241 (0%)]	Loss: 0.008472
Train Epoch: 6 [4000/84241 (0%)]	Loss: 0.007525
Train Epoch: 6 [5000/84241 (0%)]	Loss: 0.007880
Train Epoch: 6 [6000/84241 (0%)]	Loss: 0.007148
Train Epoch: 6 [7000/84241 (0%)]	Loss: 0.008294
Train Epoch: 6 [8000/84241 (0%)]	Loss: 0.008921
Train Epoch: 6 [9000/84241 (0%)]	Loss: 0.007231
Train Epoch: 6 [10000/84241 (0%)]	Loss: 0.006819
Train Epoch: 6 [11000/84241 (0%)]	Loss: 0.010620
Train Epoch: 6 [12000/84241 (0%)]	Loss: 0.006924
Train Epoch: 6 [13000/84241 (0%)]	Loss: 0.007024
Train Epoch: 6 [14000/84241 (0%)]	Loss: 0.013462
Train Epoch: 6 [15000/84241 (0%)]	Loss: 0.006874
Train Epoch: 6 [16000/84241 (0%)]	Loss: 0.007283
Train Epoch: 6 [17000/84241 (0%)]	Loss: 0.007826
Train Epoch: 6 [18000/84241 (0%)]	Loss: 0.007421
Train Epoch: 6 [19000/84241 (0%)]	Lo

...

Train Epoch: 39 [52000/84241 (0%)]	Loss: 0.008239
Train Epoch: 39 [53000/84241 (0%)]	Loss: 0.006979
Train Epoch: 39 [54000/84241 (0%)]	Loss: 0.006862
Train Epoch: 39 [55000/84241 (0%)]	Loss: 0.006977
Train Epoch: 39 [56000/84241 (0%)]	Loss: 0.007643
Train Epoch: 39 [57000/84241 (0%)]	Loss: 0.012796
Train Epoch: 39 [58000/84241 (0%)]	Loss: 0.006859
Train Epoch: 39 [59000/84241 (0%)]	Loss: 0.007175
Train Epoch: 39 [60000/84241 (0%)]	Loss: 0.008304
Train Epoch: 39 [61000/84241 (0%)]	Loss: 0.007779
Train Epoch: 39 [62000/84241 (0%)]	Loss: 0.007320
Train Epoch: 39 [63000/84241 (0%)]	Loss: 0.007052
Train Epoch: 39 [64000/84241 (0%)]	Loss: 0.010898
Train Epoch: 39 [65000/84241 (0%)]	Loss: 0.008155
Train Epoch: 39 [66000/84241 (0%)]	Loss: 0.007581
Train Epoch: 39 [26331/84241 (0%)]	Loss: 0.009825
	Training set: Average loss: 0.0082
	Testing set: Average loss: 0.0083

Epoch 40: 
Train Epoch: 40 [0/84241 (0%)]	Loss: 0.007348
Train Epoch: 40 [1000/84241 (0%)]	Loss: 0.006963
Train Epoch: 40 [2000/

Train Epoch: 42 [11000/84241 (0%)]	Loss: 0.006561
Train Epoch: 42 [12000/84241 (0%)]	Loss: 0.007391
Train Epoch: 42 [13000/84241 (0%)]	Loss: 0.007146
Train Epoch: 42 [14000/84241 (0%)]	Loss: 0.007044
Train Epoch: 42 [15000/84241 (0%)]	Loss: 0.007747
Train Epoch: 42 [16000/84241 (0%)]	Loss: 0.008422
Train Epoch: 42 [17000/84241 (0%)]	Loss: 0.007550
Train Epoch: 42 [18000/84241 (0%)]	Loss: 0.008294
Train Epoch: 42 [19000/84241 (0%)]	Loss: 0.007077
Train Epoch: 42 [20000/84241 (0%)]	Loss: 0.007610
Train Epoch: 42 [21000/84241 (0%)]	Loss: 0.007609
Train Epoch: 42 [22000/84241 (0%)]	Loss: 0.010526
Train Epoch: 42 [23000/84241 (0%)]	Loss: 0.006435
Train Epoch: 42 [24000/84241 (0%)]	Loss: 0.007996
Train Epoch: 42 [25000/84241 (0%)]	Loss: 0.007541
Train Epoch: 42 [26000/84241 (0%)]	Loss: 0.007225
Train Epoch: 42 [27000/84241 (0%)]	Loss: 0.006941
Train Epoch: 42 [28000/84241 (0%)]	Loss: 0.007584
Train Epoch: 42 [29000/84241 (0%)]	Loss: 0.006849
Train Epoch: 42 [30000/84241 (0%)]	Loss: 0.007246


Train Epoch: 44 [42000/84241 (0%)]	Loss: 0.009004
Train Epoch: 44 [43000/84241 (0%)]	Loss: 0.007374
Train Epoch: 44 [44000/84241 (0%)]	Loss: 0.007966
Train Epoch: 44 [45000/84241 (0%)]	Loss: 0.006894
Train Epoch: 44 [46000/84241 (0%)]	Loss: 0.010947
Train Epoch: 44 [47000/84241 (0%)]	Loss: 0.006865
Train Epoch: 44 [48000/84241 (0%)]	Loss: 0.006333
Train Epoch: 44 [49000/84241 (0%)]	Loss: 0.007071
Train Epoch: 44 [50000/84241 (0%)]	Loss: 0.006856
Train Epoch: 44 [51000/84241 (0%)]	Loss: 0.010613
Train Epoch: 44 [52000/84241 (0%)]	Loss: 0.007618
Train Epoch: 44 [53000/84241 (0%)]	Loss: 0.007424
Train Epoch: 44 [54000/84241 (0%)]	Loss: 0.006708
Train Epoch: 44 [55000/84241 (0%)]	Loss: 0.007502
Train Epoch: 44 [56000/84241 (0%)]	Loss: 0.006771
Train Epoch: 44 [57000/84241 (0%)]	Loss: 0.006753
Train Epoch: 44 [58000/84241 (0%)]	Loss: 0.007169
Train Epoch: 44 [59000/84241 (0%)]	Loss: 0.006603
Train Epoch: 44 [60000/84241 (0%)]	Loss: 0.008038
Train Epoch: 44 [61000/84241 (0%)]	Loss: 0.009414


	Testing set: Average loss: 0.0082

Epoch 47: 
Train Epoch: 47 [0/84241 (0%)]	Loss: 0.007921
Train Epoch: 47 [1000/84241 (0%)]	Loss: 0.007152
Train Epoch: 47 [2000/84241 (0%)]	Loss: 0.007298
Train Epoch: 47 [3000/84241 (0%)]	Loss: 0.006924
Train Epoch: 47 [4000/84241 (0%)]	Loss: 0.010994
Train Epoch: 47 [5000/84241 (0%)]	Loss: 0.007474
Train Epoch: 47 [6000/84241 (0%)]	Loss: 0.008134
Train Epoch: 47 [7000/84241 (0%)]	Loss: 0.007504
Train Epoch: 47 [8000/84241 (0%)]	Loss: 0.007091
Train Epoch: 47 [9000/84241 (0%)]	Loss: 0.007299
Train Epoch: 47 [10000/84241 (0%)]	Loss: 0.009056
Train Epoch: 47 [11000/84241 (0%)]	Loss: 0.007850
Train Epoch: 47 [12000/84241 (0%)]	Loss: 0.008407
Train Epoch: 47 [13000/84241 (0%)]	Loss: 0.009596
Train Epoch: 47 [14000/84241 (0%)]	Loss: 0.007167
Train Epoch: 47 [15000/84241 (0%)]	Loss: 0.006972
Train Epoch: 47 [16000/84241 (0%)]	Loss: 0.007050
Train Epoch: 47 [17000/84241 (0%)]	Loss: 0.006762
Train Epoch: 47 [18000/84241 (0%)]	Loss: 0.006895
Train Epoch: 47 

Train Epoch: 49 [28000/84241 (0%)]	Loss: 0.006355
Train Epoch: 49 [29000/84241 (0%)]	Loss: 0.006849
Train Epoch: 49 [30000/84241 (0%)]	Loss: 0.006684
Train Epoch: 49 [31000/84241 (0%)]	Loss: 0.007563
Train Epoch: 49 [32000/84241 (0%)]	Loss: 0.007298
Train Epoch: 49 [33000/84241 (0%)]	Loss: 0.011331
Train Epoch: 49 [34000/84241 (0%)]	Loss: 0.008974
Train Epoch: 49 [35000/84241 (0%)]	Loss: 0.006941
Train Epoch: 49 [36000/84241 (0%)]	Loss: 0.007883
Train Epoch: 49 [37000/84241 (0%)]	Loss: 0.006518
Train Epoch: 49 [38000/84241 (0%)]	Loss: 0.007387
Train Epoch: 49 [39000/84241 (0%)]	Loss: 0.010074
Train Epoch: 49 [40000/84241 (0%)]	Loss: 0.007129
Train Epoch: 49 [41000/84241 (0%)]	Loss: 0.007179
Train Epoch: 49 [42000/84241 (0%)]	Loss: 0.007999
Train Epoch: 49 [43000/84241 (0%)]	Loss: 0.010402
Train Epoch: 49 [44000/84241 (0%)]	Loss: 0.007623
Train Epoch: 49 [45000/84241 (0%)]	Loss: 0.009749
Train Epoch: 49 [46000/84241 (0%)]	Loss: 0.011573
Train Epoch: 49 [47000/84241 (0%)]	Loss: 0.006948


9.757140159606934
0.831522876024247


-2.0118825435638428

In [ ]:
from __future__ import print_function, division


train_loss_file = 'results/train_loss_split_{}.txt'.format(null_split)
f = open(train_loss_file, 'w')
f.close()
validate_loss_file = 'results/validate_loss_split_{}.txt'.format(null_split)
f = open(validate_loss_file, 'w')
f.close()
    train_accuracy_file = 'results/train_accuracy_split_{}.txt'.format(null_split)
    f = open(train_accuracy_file, 'w')
    f.close()
    validate_accuracy_file = 'results/validate_accuracy_split_{}.txt'.format(null_split)
    f = open(validate_accuracy_file, 'w')
    f.close()

    model_file = 'models/saved_model_split_{}'.format(null_split)

    for epoch in range(1, 51):
        print('\nEpoch %d: ' % epoch)
        loss, accuracy = train(model, device, train_loader, optimizer)

        with open(train_loss_file, "a") as file:
            file.write(str(loss))
            file.write('\n')
        with open(train_accuracy_file, "a") as file:
            file.write(str(accuracy))
            file.write('\n')

        loss, accuracy = test(model, device, test_loader)

        with open(validate_loss_file, "a") as file:
            file.write(str(loss))
            file.write('\n')
        with open(validate_accuracy_file, "a") as file:
            file.write(str(accuracy))
            file.write('\n')

        if epoch % 5 == 0:
            with open(model_file, 'wb') as f:
                torch.save(model.state_dict(), f)
